In [1]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import Literal

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = 'url'
    PPTX = 'pptx'
    DOCX = 'docx'
    XLS = 'xls'
    XLSX = 'xlsx'
    XML = 'xml'
    GDOC = 'gdoc'
    GSHEET = 'gsheet'
    GSLIDE = 'gslide'
    GPDF = 'gpdf'
    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'


In [2]:
from pydantic import BaseModel, Field
from typing import Literal, List, Any

class RubricGeneratorArgs(BaseModel):
    grade_level: Literal["elementary", "middle", "high", "college", "professional"] = Field(..., description="Educational level to which the content is directed")
    point_scale: str = Field(..., description="Point scale used for the assignment (e.g., 100-point scale, 4.0 GPA scale)")
    standard: str = Field(..., description="Educational standard or guideline being followed")
    file_type: str = Field(..., description="Type of file being handled, according to the defined enumeration")
    file_url: str = Field(..., description="URL or path of the file to be processed")
    lang: str = Field(..., description="Language in which the file or content is written")

In [3]:
class ScoreRange(BaseModel):
    label: str = Field(..., description="Label for the score range (e.g., 'Excellent', 'Good')")
    min_score: int = Field(..., description="Minimum score for this label")
    max_score: int = Field(..., description="Maximum score for this label")
    description: str = Field(..., description="Description for this score range")

class RubricCriterion(BaseModel):
    criterion_name: str = Field(..., description="Name of the criterion (e.g., 'Model Implementation')")
    score: Any = Field(..., description="Score for the criterion, can be a number, grade, or any value depending on rubric definition")
    score_range: List[ScoreRange] = Field(..., description="List of possible score ranges with their respective labels and descriptions")
    description: str = Field(..., description="Description of performance at the given score")

class Rubric(BaseModel):
    criteria: List[RubricCriterion] = Field(..., description="List of all criteria evaluated in the rubric")
    total_score: Any = Field(..., description="Total score achieved across all criteria, flexible for different scoring systems")
    max_total_score: Any = Field(..., description="Maximum possible total score across all criteria, flexible for different scoring systems")

In [4]:
class FileHandlerError(Exception):
    """Raised when a file content cannot be loaded. Used for tools which require file handling."""
    def __init__(self, message, url=None):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class ImageHandlerError(Exception):
    """Raised when an image cannot be loaded. Used for tools which require image handling."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

class VideoTranscriptError(Exception):
    """Raised when a video transcript cannot be loaded. Used for tools which require video transcripts."""
    def __init__(self, message, url):
        self.message = message
        self.url = url
        super().__init__(self.message)

    def __str__(self):
        return f"{self.message}"

In [5]:
%%writefile document_loaders.py
from langchain_community.document_loaders import YoutubeLoader, PyPDFLoader, TextLoader, UnstructuredURLLoader, UnstructuredPowerPointLoader, Docx2txtLoader, UnstructuredExcelLoader, UnstructuredXMLLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import os
import tempfile
import uuid
import requests
import gdown
from enum import Enum

class FileType(Enum):
    PDF = 'pdf'
    CSV = 'csv'
    TXT = 'txt'
    MD = 'md'
    URL = 'url'
    PPTX = 'pptx'
    DOCX = 'docx'
    XLS = 'xls'
    XLSX = 'xlsx'
    XML = 'xml'
    GDOC = 'gdoc'
    GSHEET = 'gsheet'
    GSLIDE = 'gslide'
    GPDF = 'gpdf'
    YOUTUBE_URL = 'youtube_url'
    IMG = 'img'

STRUCTURED_TABULAR_FILE_EXTENSIONS = {"csv", "xls", "xlsx", "gsheet", "xml"}

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

def read_text_file(file_path):
    # Get the directory containing the script file
    script_dir = os.path.dirname(os.path.abspath(__file__))

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    with open(absolute_file_path, 'r') as file:
        return file.read()

def get_docs(file_url: str, file_type: str, verbose=True):
    file_type = file_type.lower()
    try:
        file_loader = file_loader_map[FileType(file_type)]
        docs = file_loader(file_url, verbose)

        return docs

    except Exception as e:
        print(e)
        print(f"Unsupported file type: {file_type}")
        raise FileHandlerError(f"Unsupported file type", file_url) from e

class FileHandler:
    def __init__(self, file_loader, file_extension):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):
        # Generate a unique filename with a UUID prefix
        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        # Download the file from the URL and save it to a temporary file
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful

        with tempfile.NamedTemporaryFile(delete=False, prefix=unique_filename) as temp_file:
            temp_file.write(response.content)
            temp_file_path = temp_file.name

        # Use the file_loader to load the documents
        try:
            loader = self.file_loader(file_path=temp_file_path)
        except Exception as e:
            print(f"No such file found at {temp_file_path}")
            raise FileHandlerError(f"No file found", temp_file_path) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available", temp_file_path) from e

        # Remove the temporary file
        os.remove(temp_file_path)

        return documents

def load_pdf_documents(pdf_url: str, verbose=False):
    pdf_loader = FileHandler(PyPDFLoader, "pdf")
    docs = pdf_loader.load(pdf_url)

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PDF file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_csv_documents(csv_url: str, verbose=False):
    csv_loader = FileHandler(CSVLoader, "csv")
    docs = csv_loader.load(csv_url)

    if docs:
        if verbose:
            print(f"Found CSV file")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_txt_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "txt")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found TXT file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_md_documents(notes_url: str, verbose=False):
    notes_loader = FileHandler(TextLoader, "md")
    docs = notes_loader.load(notes_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found MD file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_url_documents(url: str, verbose=False):
    url_loader = UnstructuredURLLoader(urls=[url])
    docs = url_loader.load()

    if docs:
        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found URL")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_pptx_documents(pptx_url: str, verbose=False):
    pptx_handler = FileHandler(UnstructuredPowerPointLoader, 'pptx')

    docs = pptx_handler.load(pptx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found PPTX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_docx_documents(docx_url: str, verbose=False):
    docx_handler = FileHandler(Docx2txtLoader, 'docx')
    docs = docx_handler.load(docx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found DOCX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xls_documents(xls_url: str, verbose=False):
    xls_handler = FileHandler(UnstructuredExcelLoader, 'xls')
    docs = xls_handler.load(xls_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLS file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xlsx_documents(xlsx_url: str, verbose=False):
    xlsx_handler = FileHandler(UnstructuredExcelLoader, 'xlsx')
    docs = xlsx_handler.load(xlsx_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XLSX file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_xml_documents(xml_url: str, verbose=False):
    xml_handler = FileHandler(UnstructuredXMLLoader, 'xml')
    docs = xml_handler.load(xml_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found XML file")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

class FileHandlerForGoogleDrive:
    def __init__(self, file_loader, file_extension='docx'):
        self.file_loader = file_loader
        self.file_extension = file_extension

    def load(self, url):

        unique_filename = f"{uuid.uuid4()}.{self.file_extension}"

        try:
            gdown.download(url=url, output=unique_filename, fuzzy=True)
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        try:
            loader = self.file_loader(file_path=unique_filename)
        except Exception as e:
            print(f"No such file found at {unique_filename}")
            raise FileHandlerError(f"No file found", unique_filename) from e

        try:
            documents = loader.load()
        except Exception as e:
            print(f"File content might be private or unavailable or the URL is incorrect.")
            raise FileHandlerError(f"No file content available") from e

        os.remove(unique_filename)

        return documents

def load_gdocs_documents(drive_folder_url: str, verbose=False):

    gdocs_loader = FileHandlerForGoogleDrive(Docx2txtLoader)

    docs = gdocs_loader.load(drive_folder_url)

    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Docs files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gsheets_documents(drive_folder_url: str, verbose=False):
    gsheets_loader = FileHandlerForGoogleDrive(UnstructuredExcelLoader, 'xlsx')
    docs = gsheets_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Sheets files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gslides_documents(drive_folder_url: str, verbose=False):
    gslides_loader = FileHandlerForGoogleDrive(UnstructuredPowerPointLoader, 'pptx')
    docs = gslides_loader.load(drive_folder_url)
    if docs:

        split_docs = splitter.split_documents(docs)

        if verbose:
            print(f"Found Google Slides files")
            print(f"Splitting documents into {len(split_docs)} chunks")

        return split_docs

def load_gpdf_documents(drive_folder_url: str, verbose=False):

    gpdf_loader = FileHandlerForGoogleDrive(PyPDFLoader,'pdf')

    docs = gpdf_loader.load(drive_folder_url)
    if docs:

        if verbose:
            print(f"Found Google PDF files")
            print(f"Splitting documents into {len(docs)} chunks")

        return docs

def load_docs_youtube_url(youtube_url: str, verbose=True) -> str:
    try:
        loader = YoutubeLoader.from_youtube_url(youtube_url, add_video_info=True)
    except Exception as e:
        print(f"No such video found at {youtube_url}")
        raise VideoTranscriptError(f"No video found", youtube_url) from e

    try:
        docs = loader.load()
        length = docs[0].metadata["length"]
        title = docs[0].metadata["title"]

    except Exception as e:
        print(f"Video transcript might be private or unavailable in 'en' or the URL is incorrect.")
        raise VideoTranscriptError(f"No video transcripts available", youtube_url) from e

    if verbose:
        print(f"Found video with title: {title} and length: {length}")
        print(f"Combined documents into a single string.")
        print(f"Beginning to process transcript...")

    split_docs = splitter.split_documents(docs)

    return split_docs

llm_for_img = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def generate_docs_from_img(img_url, verbose: bool=False):
    message = HumanMessage(
    content=[
            {
                "type": "text",
                "text": "Give me a summary of what you see in the image. It must be 3 detailed paragraphs.",
            },
            {"type": "image_url", "image_url": img_url},
        ]
    )

    try:
        response = llm_for_img.invoke([message]).content
        print(f"Generated summary: {response}")
        docs = Document(page_content=response, metadata={"source": img_url})
        split_docs = splitter.split_documents([docs])
    except Exception as e:
        print(f"Error processing the request due to Invalid Content or Invalid Image URL")
        raise ImageHandlerError(f"Error processing the request", img_url) from e

    return split_docs

file_loader_map = {
    FileType.PDF: load_pdf_documents,
    FileType.CSV: load_csv_documents,
    FileType.TXT: load_txt_documents,
    FileType.MD: load_md_documents,
    FileType.URL: load_url_documents,
    FileType.PPTX: load_pptx_documents,
    FileType.DOCX: load_docx_documents,
    FileType.XLS: load_xls_documents,
    FileType.XLSX: load_xlsx_documents,
    FileType.XML: load_xml_documents,
    FileType.GDOC: load_gdocs_documents,
    FileType.GSHEET: load_gsheets_documents,
    FileType.GSLIDE: load_gslides_documents,
    FileType.GPDF: load_gpdf_documents,
    FileType.YOUTUBE_URL: load_docs_youtube_url,
    FileType.IMG: generate_docs_from_img
}

Overwriting document_loaders.py


In [6]:
!pip install langchain langchain-core langchain-google-genai langchain_community langchain-chroma chroma

In [7]:
!pip install pypdf fpdf youtube-transcript-api pytube unstructured python-pptx docx2txt networkx pandas xlrd openpyxl gdown pytest PyPDF2 psutil

In [8]:
!pip uninstall -y nltk
!pip install nltk
import nltk
nltk.download('punkt')

Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from typing import List, Dict
import os

from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
def read_text_file(file_path):
    # Get the current working directory
    script_dir = os.getcwd()

    # Combine the script directory with the relative file path
    absolute_file_path = os.path.join(script_dir, file_path)

    try:
        with open(absolute_file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        # Handle the case where the file is not found
        print(f"File not found: {absolute_file_path}")
        return None

In [11]:
class RubricGenerator:
    def __init__(self, args=None, vectorstore_class=Chroma, prompt=None, embedding_model=None, model=None, parser=None, verbose=False):
        default_config = {
            "model": GoogleGenerativeAI(model="gemini-1.5-flash"),
            "embedding_model": GoogleGenerativeAIEmbeddings(model='models/embedding-001'),
            "parser": JsonOutputParser(pydantic_object=Rubric),
            "prompt": read_text_file("prompt/rubric-generator-cot-prompt.txt"),
            "vectorstore_class": Chroma
        }

        self.prompt = prompt or default_config["prompt"]
        self.model = model or default_config["model"]
        self.parser = parser or default_config["parser"]
        self.embedding_model = embedding_model or default_config["embedding_model"]

        self.vectorstore_class = vectorstore_class or default_config["vectorstore_class"]
        self.vectorstore, self.retriever, self.runner = None, None, None
        self.args = args
        self.verbose = verbose

        if vectorstore_class is None: raise ValueError("Vectorstore must be provided")
        if args.grade_level is None: raise ValueError("Grade Level must be provided")
        if args.point_scale is None: raise ValueError("Point Scale must be provided")
        if args.standard is None: raise ValueError("Standard must be provided")
        if args.lang is None: raise ValueError("Language must be provided")


    def compile(self, documents: List[Document]):
        # Return the chain
        prompt = PromptTemplate(
            template=self.prompt,
            input_variables=["attribute_collection"],
            partial_variables={"format_instructions": self.parser.get_format_instructions()}
        )

        if self.runner is None:
            print(f"Creating vectorstore from {len(documents)} documents") if self.verbose else None
            self.vectorstore = self.vectorstore_class.from_documents(documents, self.embedding_model)
            print(f"Vectorstore created") if self.verbose else None

            self.retriever = self.vectorstore.as_retriever()
            print(f"Retriever created successfully") if self.verbose else None

            self.runner = RunnableParallel(
                {"context": self.retriever,
                "attribute_collection": RunnablePassthrough()
                }
            )

        chain = self.runner | prompt | self.model | self.parser

        if self.verbose: print(f"Chain compilation complete")

        return chain

    def generate_rubric(self, documents: List[Document]):
        if self.verbose: print(f"Creating the Rubric")

        chain = self.compile(documents)

        response = chain.invoke(f"Grade Level: {self.args.grade_level}, Point Scale: {self.args.point_scale}, Standard: {self.args.standard}, Language (YOU MUST RESPOND IN THIS LANGUAGE): {self.args.lang}")

        if self.verbose: print(f"Deleting vectorstore")
        self.vectorstore.delete_collection()

        return response

In [12]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [13]:
from document_loaders import get_docs

input_data = RubricGeneratorArgs(
    grade_level="professional",
    point_scale="100-point scale",
    standard="To make an ensemble Machine Learning model",
    file_url="https://raw.github.com/AaronSosaRamos/mission-flights/main/files-for-test/sample_input.pdf",
    file_type="pdf",
    lang="en"
)


docs = get_docs(input_data.file_url, input_data.file_type, True)

Found PDF file
Splitting documents into 5 chunks


In [14]:
docs

[Document(metadata={'source': '/tmp/5334ff74-cbb7-414a-94eb-a9d14515ce27.pdflknwpthf', 'page': 0}, page_content='Objectiv e:\nStudentsaretaskedwithbuildinganensemblelearningmodeltopredictflightdelaytimesbasedonflightdata.Theensemblewillconsistofmodelsbuiltfromscratch,includingSupportVectorMachine(SVM),DecisionTrees,andLogisticRegression,usingonlypandasandnumpy.\nDatasetOverview:\nYouwillbeprovidedwithadatasetofoutboundflightsfrom2023-2024forLahore(LHE),Karachi(KHI),andIslamabad(ISB)airports.Thedatasetincludesthefollowingfields:\n●Departuretime\n●Estimateddeparturetime\n●Delaytime(inminutes)\n●Arrivaldetails(destination,time,etc.)\n●AirlineandflightinformationTask:\n1.TargetVariable:\n○Thetargetvariableisdelay_time,whichrepresentsthedelayindeparturetimeinminutes.\n○Binning:Youwillbinthedelay_timeintoatotalof8bins.2.EnsembleModel:\n○Youarerequiredtobuildanensemblelearningmodelfromscratch.\n○Useatleastthreemodelsaslearners:SupportVectorMachine(SVM),DecisionTrees,andLogisticRegression.\n○I

In [15]:
%%writefile rubric-generator-cot-prompt.txt
You are an expert in generating educational assignments resistant to AI tools. Generate three AI-resistant versions of the uploaded assignment while maintaining core educational objectives. Each version should challenge AI tools by incorporating nuanced questions, promoting original thought, or solving complex problems. Explain briefly how each version increases AI resistance using **Chain-of-Thought (CoT)** reasoning.

Here are the grade level, point scale, standard, and language of the assignment:
{attribute_collection}

Next, generate a rubric based on the provided inputs:
- Align the rubric with the grade level, point scale, and standard.
- Provide clear grading criteria for each aspect of the assignment with point allocations matching the provided scale.

**Follow this steps**:
- **Step 1**: Analyze the topic, grade level, and standard.
- **Step 2**: Identify how AI tools could solve the original assignment easily.
- **Step 3**: Introduce modifications that require higher-order thinking, ambiguity, or real-world application to resist AI-generated responses.
- **Step 4**: Explain how each modification adds complexity for AI.

Your response should follow this format:
{format_instructions}

Writing rubric-generator-cot-prompt.txt


In [16]:
output = RubricGenerator(args=input_data, verbose=True).generate_rubric(docs)

Creating the Rubric
Creating vectorstore from 5 documents
Vectorstore created
Retriever created successfully
Chain compilation complete
Deleting vectorstore


In [17]:
output

{'criteria': [{'criterion_name': 'Ensemble Model Design',
   'score': None,
   'score_range': [{'label': 'Excellent',
     'min_score': 80,
     'max_score': 100,
     'description': 'Demonstrates a thorough understanding of ensemble techniques and selects appropriate models for the task, considering the strengths and weaknesses of each model. The design is well-justified and addresses the specific challenges of the dataset.'},
    {'label': 'Good',
     'min_score': 60,
     'max_score': 79,
     'description': "Demonstrates a good understanding of ensemble techniques and selects suitable models. The design could be strengthened by further justification and consideration of the dataset's characteristics."},
    {'label': 'Needs Improvement',
     'min_score': 40,
     'max_score': 59,
     'description': 'Demonstrates a basic understanding of ensemble techniques but the model selection and design lack clarity and justification. The design may not be well-suited for the task.'},
    {'